# Importing libraries

In [44]:
import pandas as pd     # for calculation
import numpy as np

from sklearn.experimental import enable_iterative_imputer   # for mssing values calculation
from sklearn.impute import IterativeImputer
from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt     # for graph

from sklearn.cluster import DBSCAN  # for outliers

# User-Defined Function (UDF)

In [45]:
# creating a function for monthly calculation from yearly and weakly 
def monthly(x, value):
    if x in [409,419, 619, 649]:
        return (value * 30)/ 365
    elif x in [169, 219, 239, 249, 189, 269, 279, 199, 289, 299, 309, 319, 329]:
        return (value * 30)/ 7
    else:
        return value

In [46]:
# creating a function for sector and state spliting from Common-id
def split_string(s):
    part0 = s[:13]   # first 13 bytes
    part1 = s[13:14]  # next 2 bytes
    part2 = s[14:16]  # next 2 bytes
    part3 = s[16:]   # remaining bytes
    return pd.Series([part0,part1, part2, part3]) 

# Importing data

In [47]:
# Importing level 2 data from divice
col_specification=[(0,38),(38,39),(39,41),(41,43),(43,44),(44,45),(45,48),(48,49),(49,51),(51,53),(53,54),(54,56),(56,57),(57,59),(59,61),(61,63),(63,65),(65,67),(67,68),(68,69),(69,84)]
col_names=['Common-ID','Questionnaire No.','Level','Person Srl No.','Relation to head (code)','Gender','Age(in years)','Marital Status (code)','Highest educational level attained (code)','Total year of education completed','Whether used internet from any location during last 30 days','No. of days stayed away from home during last 30 days','No. of meals usually taken in a day','No. of meals taken during last 30 days from school, balwadi etc.','No. of meals taken during last 30 days  others','No. of meals taken during last 30 days from employer as perquisites or part of wage','No. of meals taken during last 30 days at home','No. of meals taken during last 30 days on payment','FDQ original member(generated field)','Status of Member as on revisit','Multiplier']
level_2 = pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_02.txt', header=None, colspecs=col_specification, names = col_names)

# As provided in "Read me" text dividing multiplier by 100
level_2['Multiplier'] = level_2['Multiplier']/100
level_2.head(5)

,Common-ID,Questionnaire No.,Level,Person Srl No.,Relation to head (code),Gender,Age(in years),Marital Status (code),Highest educational level attained (code),Total year of education completed,...,No. of days stayed away from home during last 30 days,No. of meals usually taken in a day,"No. of meals taken during last 30 days from school, balwadi etc.",No. of meals taken during last 30 days others,No. of meals taken during last 30 days from employer as perquisites or part of wage,No. of meals taken during last 30 days at home,No. of meals taken during last 30 days on payment,FDQ original member(generated field),Status of Member as on revisit,Multiplier
0,HCES2022655561010131113011 101202 201,H,2,1,1,1,48,2,6,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,58.0,1,1.0,355.6
1,HCES2022655561010131113011 101202 201,H,2,2,2,2,46,2,1,NaN,...,0.0,2.0,NaN,NaN,NaN,NaN,60.0,1,1.0,355.6
2,HCES2022655561010131113011 101202 201,H,2,3,5,1,24,1,13,18.0,...,0.0,2.0,NaN,NaN,NaN,NaN,58.0,1,1.0,355.6
3,HCES2022655561010131113011 101202 201,H,2,4,5,1,18,1,7,13.0,...,0.0,2.0,NaN,NaN,NaN,NaN,56.0,1,1.0,355.6
4,HCES2022655561010131113011 101202 201,H,2,5,5,2,21,1,12,17.0,...,0.0,2.0,NaN,NaN,NaN,NaN,54.0,1,1.0,355.6


In [48]:
z =level_2.groupby('Common-ID')['Multiplier'].unique().reset_index()

In [49]:
z.head()

,Common-ID,Multiplier
0,HCES2022310002282831212025 228111 301,[964.98]
1,HCES2022310002282831212025 228111 302,[964.98]
2,HCES2022310002282831212025 228111 303,[964.98]
3,HCES2022310002282831212025 228111 304,[964.98]
4,HCES2022310002282831212025 228111 305,[964.98]


In [50]:
z[['part2','Sector', 'State', 'part3']] = z['Common-ID'].apply(split_string)

In [51]:
z.head()

,Common-ID,Multiplier,part2,Sector,State,part3
0,HCES2022310002282831212025 228111 301,[964.98],HCES202231000,2,28,2831212025 228111 301
1,HCES2022310002282831212025 228111 302,[964.98],HCES202231000,2,28,2831212025 228111 302
2,HCES2022310002282831212025 228111 303,[964.98],HCES202231000,2,28,2831212025 228111 303
3,HCES2022310002282831212025 228111 304,[964.98],HCES202231000,2,28,2831212025 228111 304
4,HCES2022310002282831212025 228111 305,[964.98],HCES202231000,2,28,2831212025 228111 305


In [52]:
# Importing level-14 data from device
col_specification=[(0,38),(38,39),(39,41),(41,46),(46,49),(49,59),(59,74)]
col_names=['Common-ID','Questionnaire No.','Level','Section','Item Code','Value (in Rs)','Multiplier']
lvl_14= pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_14.txt', header=None, colspecs=col_specification, names = col_names)

# As provided in "Read me" text dividing multiplier by 100
lvl_14['Multiplier'] = lvl_14['Multiplier']/100
lvl_14.head(5)

,Common-ID,Questionnaire No.,Level,Section,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,14,5.1,139,90,88.9
1,HCES2022655621010121713017 101111 310,F,14,5.1,139,90,88.9
2,HCES2022655621010121713017 101111 311,F,14,5.1,139,25,88.9
3,HCES2022655621010121713017 101111 313,F,14,5.1,139,45,88.9
4,HCES2022655371010122023014 201111 202,F,14,5.1,139,75,1405.0


### Primary cleanning
Splitting the non-usable column and multiplier divided by 100 as defined in the "read me" word file

In [53]:
# for counting total number of members of a household
lvl_2 = level_2.value_counts('Common-ID').to_frame().reset_index()
lvl_2.head(5)

,Common-ID,count
0,HCES2022666371080811523175 108112 107,37
1,HCES2022341462363612342036 236101 1312,31
2,HCES2022636211090934223383 109205 303,30
3,HCES2022670621353510123017 235101 203,29
4,HCES2022636811090920723403 109125 308,28


In [54]:
mu = pd.merge(lvl_2,z, on='Common-ID',how='left')
mu.head()

,Common-ID,count,Multiplier,part2,Sector,State,part3
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308


In [55]:
mu['total_mambers'] = mu['count']*mu['Multiplier']
mu.head(5)

,Common-ID,count,Multiplier,part2,Sector,State,part3,total_mambers
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,[68559.89]
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,[57642.020000000004]
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,[62617.5]
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,[2557.5099999999998]
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,[39055.799999999996]


In [56]:
# Dropping level and Section from data frame (lvl_14)
level_14 = lvl_14.drop(['Level','Section'],axis=1)
level_14.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,139,90,88.9
1,HCES2022655621010121713017 101111 310,F,139,90,88.9
2,HCES2022655621010121713017 101111 311,F,139,25,88.9
3,HCES2022655621010121713017 101111 313,F,139,45,88.9
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0


main_data = pd.merge(level_14,mu, on='Common-ID', how='left')
main_data.head(5)

level_14.count()

## Appling monthly function
so we can read monthly expendeture

In [57]:
# Creating a new column for monthly calculated values using monthly function
level_14['Monthly Value'] = level_14.apply(lambda row: monthly(row['Item Code'], row['Value (in Rs)']), axis=1)
level_14.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,90.0
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,90.0
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,25.0
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,45.0
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,75.0


# One item study
For checking data is representing correctly or not we only study 'Item code' 169 for milk and milk products 

In [58]:
# Filtering the data-frame for checking milk & milk prodect consumption in a single housee hold
##checking if monthly funtion is functioning properly
todoroki = level_14[level_14['Item Code'].isin([409])]
temp1 = todoroki.copy()
temp1.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
3649908,HCES2022655561010131113011 101202 201,C,409,24050,355.60,1976.712329
3649915,HCES2022655561010131113011 101202 301,C,409,30500,303.31,2506.849315
3649922,HCES2022655561010131113011 101202 302,C,409,27100,303.31,2227.397260
3649929,HCES2022655561010131113011 101202 303,C,409,16250,303.31,1335.616438
3649954,HCES2022655561010131113011 101202 307,C,409,22800,303.31,1873.972603


In [59]:
main_data = pd.merge(mu,temp1, on='Common-ID', how='left')
main_data.head(5)

,Common-ID,count,Multiplier_x,part2,Sector,State,part3,total_mambers,Questionnaire No.,Item Code,Value (in Rs),Multiplier_y,Monthly Value
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,[68559.89],C,409.0,375000.0,1852.97,30821.917808
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,[57642.020000000004],C,409.0,66679.0,1859.42,5480.465753
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,[62617.5],C,409.0,41500.0,2087.25,3410.958904
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,[2557.5099999999998],C,409.0,7980.0,88.19,655.890411
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,[39055.799999999996],NaN,NaN,NaN,NaN,NaN


In [60]:
main_data = main_data[main_data['Sector'].isin(['1'])]


In [61]:
main_data['total_mambers'] = main_data['total_mambers'].astype(int)
main_data.head(5)

,Common-ID,count,Multiplier_x,part2,Sector,State,part3,total_mambers,Questionnaire No.,Item Code,Value (in Rs),Multiplier_y,Monthly Value
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,68559,C,409.0,375000.0,1852.97,30821.917808
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,62617,C,409.0,41500.0,2087.25,3410.958904
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,2557,C,409.0,7980.0,88.19,655.890411
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,39055,NaN,NaN,NaN,NaN,NaN
5,HCES2022650161080821123081 208201 202,28,[581.65],HCES202265016,1,08,0821123081 208201 202,16286,C,409.0,119000.0,581.65,9780.821918


In [62]:
# description of monthly value only for milk & milk prodects ('4099') 
main_data['total_Value'] = main_data['Monthly Value']*main_data['Multiplier_x']
main_data.head(5)

,Common-ID,count,Multiplier_x,part2,Sector,State,part3,total_mambers,Questionnaire No.,Item Code,Value (in Rs),Multiplier_y,Monthly Value,total_Value
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,68559,C,409.0,375000.0,1852.97,30821.917808,[57112089.04109589]
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,62617,C,409.0,41500.0,2087.25,3410.958904,[7119523.97260274]
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,2557,C,409.0,7980.0,88.19,655.890411,[57842.975342465754]
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,39055,NaN,NaN,NaN,NaN,NaN,[nan]
5,HCES2022650161080821123081 208201 202,28,[581.65],HCES202265016,1,08,0821123081 208201 202,16286,C,409.0,119000.0,581.65,9780.821918,[5689015.068493151]


In [63]:
temp_dropped = main_data.drop(['Questionnaire No.','Item Code','Multiplier_x','count','Multiplier_y','part2','part3'],axis=1)
temp_dropped.head(5)

,Common-ID,Sector,State,total_mambers,Value (in Rs),Monthly Value,total_Value
0,HCES2022666371080811523175 108112 107,1,08,68559,375000.0,30821.917808,[57112089.04109589]
2,HCES2022636211090934223383 109205 303,1,09,62617,41500.0,3410.958904,[7119523.97260274]
3,HCES2022670621353510123017 235101 203,1,35,2557,7980.0,655.890411,[57842.975342465754]
4,HCES2022636811090920723403 109125 308,1,09,39055,NaN,NaN,[nan]
5,HCES2022650161080821123081 208201 202,1,08,16286,119000.0,9780.821918,[5689015.068493151]


In [75]:
#  Creating predication for missing values
#  Select the columns to impute
columns_to_impute = ['Value (in Rs)','Monthly Value','total_Value','total_mambers']

# Create an imputer object
imputer = IterativeImputer()

# Fit and transform the data, only for the selected columns
imputed_data = imputer.fit_transform(temp_dropped[columns_to_impute])

# Convert back to dataframe
imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute)

# Drop the original columns from temp2
#temp2_dropped = temp_dropped.drop(columns=columns_to_impute)

In [76]:
imputed_df.head(4)

,Value (in Rs),Monthly Value,total_Value,total_mambers
0,375000.000000,30821.917808,5.711209e+07,68559.0
1,41500.000000,3410.958904,7.119524e+06,62617.0
2,7980.000000,655.890411,5.784298e+04,2557.0
3,11168.631996,917.969753,4.310614e+06,39055.0


In [78]:
imputed_df['per_person'] = imputed_df['total_Value']/imputed_df['total_mambers']
imputed_df.head(4)

,Value (in Rs),Monthly Value,total_Value,total_mambers,per_person
0,375000.000000,30821.917808,5.711209e+07,68559.0,833.035620
1,41500.000000,3410.958904,7.119524e+06,62617.0,113.699538
2,7980.000000,655.890411,5.784298e+04,2557.0,22.621422
3,11168.631996,917.969753,4.310614e+06,39055.0,110.372916


In [79]:
imputed_df.describe()

,Value (in Rs),Monthly Value,total_Value,total_mambers,per_person
count,1.550140e+05,155014.000000,1.550140e+05,155014.000000,155014.000000
mean,1.116863e+04,917.969753,1.084451e+06,5629.527359,354.249395
std,1.651634e+04,1357.507113,2.910505e+06,7145.080849,1789.017354
min,4.000000e+00,0.328767,4.075890e+01,4.000000,0.091329
25%,3.600000e+03,295.890411,2.614972e+05,2817.000000,58.911624
50%,1.116863e+04,917.969753,7.600027e+05,5133.000000,178.802116
75%,1.116863e+04,917.969753,1.131887e+06,7464.000000,290.249542
max,1.400000e+06,115068.493151,2.202612e+08,966006.000000,135371.431684


### now for Sector '2' 

In [80]:
main_data = pd.merge(mu,temp1, on='Common-ID', how='left')
main_data.head(5)

,Common-ID,count,Multiplier_x,part2,Sector,State,part3,total_mambers,Questionnaire No.,Item Code,Value (in Rs),Multiplier_y,Monthly Value
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,[68559.89],C,409.0,375000.0,1852.97,30821.917808
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,[57642.020000000004],C,409.0,66679.0,1859.42,5480.465753
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,[62617.5],C,409.0,41500.0,2087.25,3410.958904
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,[2557.5099999999998],C,409.0,7980.0,88.19,655.890411
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,[39055.799999999996],NaN,NaN,NaN,NaN,NaN


In [81]:
len(main_data)

261746

In [82]:
main_data = main_data[main_data['Sector'].isin(['2'])]

In [83]:
main_data['total_mambers'] = main_data['total_mambers'].astype(int)
main_data.head(5)

,Common-ID,count,Multiplier_x,part2,Sector,State,part3,total_mambers,Questionnaire No.,Item Code,Value (in Rs),Multiplier_y,Monthly Value
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,57642,C,409.0,66679.0,1859.42,5480.465753
10,HCES2022350092080822132018 208101 301,25,[2809.34],HCES202235009,2,08,0822132018 208101 301,70233,C,409.0,44750.0,2809.34,3678.082192
19,HCES2022311392272750422013 227311 301,23,[1106.46],HCES202231139,2,27,2750422013 227311 301,25448,C,409.0,6190.0,1106.46,508.767123
21,HCES2022319102272712132035 227211 316,22,[1157.43],HCES202231910,2,27,2712132035 227211 316,25463,C,409.0,52500.0,1157.43,4315.068493
22,HCES2022311942272722622029 127401 301,22,[1203.09],HCES202231194,2,27,2722622029 127401 301,26467,C,409.0,39000.0,1203.09,3205.479452


In [84]:
# description of monthly value only for milk & milk prodects ('4099') 
main_data['total_Value'] = main_data['Monthly Value']*main_data['Multiplier_x']
main_data.head(5)

,Common-ID,count,Multiplier_x,part2,Sector,State,part3,total_mambers,Questionnaire No.,Item Code,Value (in Rs),Multiplier_y,Monthly Value,total_Value
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,57642,C,409.0,66679.0,1859.42,5480.465753,[10190487.631232876]
10,HCES2022350092080822132018 208101 301,25,[2809.34],HCES202235009,2,08,0822132018 208101 301,70233,C,409.0,44750.0,2809.34,3678.082192,[10332983.424657535]
19,HCES2022311392272750422013 227311 301,23,[1106.46],HCES202231139,2,27,2750422013 227311 301,25448,C,409.0,6190.0,1106.46,508.767123,[562930.4712328768]
21,HCES2022319102272712132035 227211 316,22,[1157.43],HCES202231910,2,27,2712132035 227211 316,25463,C,409.0,52500.0,1157.43,4315.068493,[4994389.726027397]
22,HCES2022311942272722622029 127401 301,22,[1203.09],HCES202231194,2,27,2722622029 127401 301,26467,C,409.0,39000.0,1203.09,3205.479452,[3856480.273972602]


In [85]:
temp_dropped = main_data.drop(['Questionnaire No.','Item Code','Multiplier_x','count','Multiplier_y','part2','part3'],axis=1)
temp_dropped.head(5)

,Common-ID,Sector,State,total_mambers,Value (in Rs),Monthly Value,total_Value
1,HCES2022341462363612342036 236101 1312,2,36,57642,66679.0,5480.465753,[10190487.631232876]
10,HCES2022350092080822132018 208101 301,2,08,70233,44750.0,3678.082192,[10332983.424657535]
19,HCES2022311392272750422013 227311 301,2,27,25448,6190.0,508.767123,[562930.4712328768]
21,HCES2022319102272712132035 227211 316,2,27,25463,52500.0,4315.068493,[4994389.726027397]
22,HCES2022311942272722622029 127401 301,2,27,26467,39000.0,3205.479452,[3856480.273972602]


In [86]:
#  Creating predication for missing values
#  Select the columns to impute
columns_to_impute = ['Value (in Rs)','Monthly Value','total_Value','total_mambers']

# Create an imputer object
imputer = IterativeImputer()

# Fit and transform the data, only for the selected columns
imputed_data = imputer.fit_transform(temp_dropped[columns_to_impute])

# Convert back to dataframe
imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute)

# Drop the original columns from temp2
#temp2_dropped = temp_dropped.drop(columns=columns_to_impute)

In [87]:
imputed_df.head(4)

,Value (in Rs),Monthly Value,total_Value,total_mambers
0,66679.0,5480.465753,1.019049e+07,57642.0
1,44750.0,3678.082192,1.033298e+07,70233.0
2,6190.0,508.767123,5.629305e+05,25448.0
3,52500.0,4315.068493,4.994390e+06,25463.0


In [88]:
imputed_df['per_person'] = imputed_df['total_Value']/imputed_df['total_mambers']
imputed_df.head(4)

,Value (in Rs),Monthly Value,total_Value,total_mambers,per_person
0,66679.0,5480.465753,1.019049e+07,57642.0,176.789279
1,44750.0,3678.082192,1.033298e+07,70233.0,147.124335
2,6190.0,508.767123,5.629305e+05,25448.0,22.120814
3,52500.0,4315.068493,4.994390e+06,25463.0,196.143020


In [89]:
imputed_df.describe()

,Value (in Rs),Monthly Value,total_Value,total_mambers,per_person
count,1.067320e+05,106732.000000,1.067320e+05,106732.000000,106732.000000
mean,2.801497e+04,2302.600387,1.820481e+06,3232.122972,768.659157
std,3.344282e+04,2748.725056,4.282086e+06,3634.810133,1422.447771
min,1.000000e+00,0.082192,3.540822e+01,8.000000,0.082205
25%,1.230000e+04,1010.958904,3.128537e+05,866.000000,227.447125
50%,2.801497e+04,2302.600387,8.856597e+05,2395.000000,553.872536
75%,2.801497e+04,2302.600387,1.968899e+06,4427.250000,760.017991
max,3.078000e+06,252986.301370,3.627697e+08,137426.000000,91065.610932


In [139]:
edu = imputed_df['per_person']
edu = edu.values.reshape(-1, 1)  # reshape to 2D array
dbscan = DBSCAN(eps=0.01, min_samples=2)
outliers = edu[dbscan.fit_predict(edu) == -1]


In [140]:
# Flatten the arrays for efficient comparison
x_flat = edu.flatten()
y_flat = outliers.flatten()

# Find the indices of values in x that are also in y
indices_to_remove = np.where(np.isin(x_flat, y_flat))[0]

# Remove those values from x
x_remaining = np.delete(x_flat, indices_to_remove)

len(x_remaining)

87440

In [136]:
# Remove outliers
per_person_without_outlier = pd.DataFrame(x_remaining)
per_person_without_outlier = per_person_without_outlier.rename(columns={0:'value'})
per_person_without_outlier.describe()

,value
count,88588.000000
mean,634.824073
std,953.168439
min,0.117420
25%,197.358392
50%,545.507189
75%,673.998313
max,37064.121247
